In [2]:
import os

with open(os.path.expanduser('~')+'/.keras/keras.json','w') as f:
    new_settings = """{\r\n
    "epsilon": 1e-07,\r\n
    "image_data_format": "channels_last",\n
    "backend": "theano",\r\n
    "floatx": "float32"\r\n
    }"""
    f.write(new_settings)

import keras

Using Theano backend.


In [3]:
import pandas as pd
import nltk
import numpy as np

In [4]:
#WordVectors

In [5]:
import gensim
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [6]:
train_DF=pd.read_csv('train.csv',index_col='id')

In [7]:
train_DF.reset_index(drop=True,inplace=True)
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [8]:
import re
numbers=re.compile(r'(\d+\.\d+|\d+)')
def cleanText(text):
    return numbers.sub('N',text)

In [54]:
texts = []
questions1=[]
questions2=[]
duplicates=[]

for idx in range(train_DF.question1.shape[0]):
    text=train_DF.question1[idx]
    text=cleanText(text)
    texts.append(text)
    sentences = nltk.tokenize.sent_tokenize(text)
    questions1.append(sentences)
    
    text=train_DF.question2[idx]
    text=cleanText(text)
    texts.append(text)
    sentences = nltk.tokenize.sent_tokenize(text)
    questions2.append(sentences)
    
    duplicates.append(train_DF.is_duplicate[idx])

In [41]:
#duplicates = to_categorical(np.asarray(duplicates))
#duplicates.shape

(404290, 2)

In [42]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten,merge
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 90278 unique tokens.


In [12]:
vocab_dim = 300 # dimensionality of your word vectors
n_symbols = len(word_index) + 1 # adding 1 to account for 0th index (for masking)
embedding_weights = np.random.random((n_symbols,vocab_dim))
for word,index in word_index.items():
    if word in model.wv.vocab:
        embedding_weights[index] = model.word_vec(word)       

In [13]:
embedding_weights.shape

(90279, 300)

In [14]:
MAX_SENT_LENGTH = 40
MAX_SENTS = 5

In [15]:
data_question1 = np.zeros((len(questions1), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
for i, sentences in enumerate(questions1):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH:
                    data_question1[i,j,k] = tokenizer.word_index[word]
                    k=k+1

In [17]:
data_question2 = np.zeros((len(questions2), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
for i, sentences in enumerate(questions2):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens =text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH:
                    data_question2[i,j,k] = tokenizer.word_index[word]
                    k=k+1

In [18]:
embedding_layer=Embedding(input_dim=len(word_index)+1,output_dim=vocab_dim,weights=[embedding_weights],
                          input_length=MAX_SENT_LENGTH,trainable=True)

In [55]:
data_question1_1000_train=data_question1[1000:2000]
data_question2_1000_train=data_question1[1000:2000]
duplicates_1000_train=duplicates[1000:2000]

In [56]:
data_question1_1000_vali=data_question1[2000:3000]
data_question2_1000_vali=data_question1[2000:3000]
duplicates_1000_vali=duplicates[2000:3000]

In [57]:
data_question1_1000_test=data_question1[:1000]
data_question2_1000_test=data_question1[:1000]
duplicates_1000_test=duplicates[:1000]

In [47]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [62]:
#Sentence Encoder for each sentence in the question
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(300))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

#Input for each Question
left_question_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
right_question_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')

#Encoders at Question Level
left_question_encoder = TimeDistributed(sentEncoder)(left_question_input)
left_lstm_sent = Bidirectional(LSTM(300))(left_question_encoder)

right_question_encoder = TimeDistributed(sentEncoder)(right_question_input)
right_lstm_sent = Bidirectional(LSTM(300))(right_question_encoder)

#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([left_lstm_sent,right_lstm_sent], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid')(both)

siamese_net = Model(input=[left_question_input,right_question_input],output=prediction)
optimizer = Adam(0.00006)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

print("model - Siamese Networks")
print(siamese_net.summary())

model - Siamese Networks
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_19 (InputLayer)            (None, 5, 40)         0                                            
____________________________________________________________________________________________________
input_20 (InputLayer)            (None, 5, 40)         0                                            
____________________________________________________________________________________________________
timedistributed_12 (TimeDistribu (None, 5, 600)        28526100    input_19[0][0]                   
____________________________________________________________________________________________________
timedistributed_13 (TimeDistribu (None, 5, 600)        28526100    input_20[0][0]                   
__________________________________________________________________

In [67]:
#data_question2_1000_train.shape
duplicates_1000_train=np.array(duplicates_1000_train).reshape(1000,1)
duplicates_1000_vali=np.array(duplicates_1000_vali).reshape(1000,1)

In [72]:
siamese_net.fit([data_question1_1000_train,data_question2_1000_train], duplicates_1000_train,
                validation_data=([data_question1_1000_vali,data_question2_1000_vali],duplicates_1000_vali)
                                  ,nb_epoch=10, batch_size=50)

ValueError: ('this shared variable already has an update expression', (backward_lstm_18_U_c, Elemwise{sub,no_inplace}.0))